In [1]:
%pip install gdown
import gdown

url = "https://drive.google.com/drive/folders/1iYO-Kwh4RFq6mm5UNPTRO7te8T6jOA6f?usp=drive_link"
gdown.download_folder(url, quiet=True)

['/content/Dataset/Dataset_resized.zip',
 '/content/Dataset/eyes_cnn.py',
 '/content/Dataset/mouth_cnn.py',
 '/content/Dataset/nose_cnn.py']

In [2]:
import os
import zipfile
import shutil
from collections import defaultdict
from sklearn.model_selection import train_test_split

# Percorso della cartella Project_Biometric/Dataset
dataset_path = "Dataset"

# Funzione per estrarre i file zip e rimuoverli
def extract_and_remove_zip(zip_path, extract_to):
    """
    Estrae il contenuto di un file zip e poi elimina il file zip.
    Args:
        zip_path (str): percorso del file zip da estrarre.
        extract_to (str): cartella di destinazione in cui estrarre i file.
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(zip_path)  # Rimuove il file zip dopo l'estrazione
    print(f"File zip {zip_path} estratto e rimosso.")

# Estrai e rimuovi i file ZIP nella cartella dataset
zip_files = [f for f in os.listdir(dataset_path) if f.endswith('.zip')]
for zip_file in zip_files:
    zip_path = os.path.join(dataset_path, zip_file)
    extract_and_remove_zip(zip_path, dataset_path)

# Funzione per analizzare il nome del file
# def parse_filename(filename):
#     parts = filename.split("_")
#     object_type = parts[0]  # Nome oggetto (es. nose, mouth, eyes)
#     age = parts[1]          # Età
#     gender = parts[2]       # Sesso
#     ethnicity = parts[3]    # Etnia
#     return object_type, age, gender, ethnicity

# dataset_path = "Dataset/Dataset_resized"

# # Processa ogni sottocartella (ad esempio eyes, mouth, nose)
# for folder in os.listdir(dataset_path):
#     folder_path = os.path.join(dataset_path, folder)

#     # Procedi solo se è una directory
#     if os.path.isdir(folder_path):
#         print(f"Elaborazione della cartella: {folder}")

#         # Crea le sottocartelle train e test
#         train_folder = os.path.join(folder_path, "train")
#         test_folder = os.path.join(folder_path, "test")
#         os.makedirs(train_folder, exist_ok=True)
#         os.makedirs(test_folder, exist_ok=True)

#         # Raggruppa i file in base a età ed etnia
#         file_groups = defaultdict(list)
#         files = [f for f in os.listdir(folder_path) if f.endswith(".jpg")]

#         # Raggruppa per età ed etnia
#         for file in files:
#             file_path = os.path.join(folder_path, file)
#             try:
#                 object_type, age, gender, ethnicity = parse_filename(file)
#                 if object_type and age and gender and ethnicity:
#                     file_groups[(age, ethnicity)].append(file)
#                 else:
#                     print(f"File non valido (parametri mancanti): {file}. Eliminazione...")
#                     os.remove(file_path)  # Elimina file non valido
#             except IndexError:
#                 print(f"File non conforme al formato: {file}. Eliminazione...")
#                 os.remove(file_path)  # Elimina file non conforme

#         # Suddividi i file in modo bilanciato tra train e test
#         train_files = []
#         test_files = []

#         for group, group_files in file_groups.items():
#             if len(group_files) == 1:
#                 # Se c'è solo un file, assegnalo al training set
#                 train_files.extend(group_files)
#             else:
#                 # Altrimenti, suddividi il gruppo in train e test (80% train, 20% test)
#                 train, test = train_test_split(group_files, test_size=0.2, random_state=42)
#                 train_files.extend(train)
#                 test_files.extend(test)

#         # Sposta i file nelle rispettive cartelle train e test
#         for file in train_files:
#             shutil.move(os.path.join(folder_path, file), os.path.join(train_folder, file))

#         for file in test_files:
#             shutil.move(os.path.join(folder_path, file), os.path.join(test_folder, file))

# # Comprime l'intera directory in un file zip
# zip_path = f"{dataset_path}.zip"
# print(f"Compressione della cartella principale in: {zip_path}")
# shutil.make_archive(dataset_path, 'zip', dataset_path)

print("Processo completato!")




File zip Dataset/Dataset_resized.zip estratto e rimosso.
Processo completato!


**Conteggio numero immagini per ogni set**

In [3]:
dataset_path = "Dataset"
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)

    # Procedi solo se è una directory
    if os.path.isdir(folder_path):
        train_folder = os.path.join(folder_path, "train")
        test_folder = os.path.join(folder_path, "test")

        train_count = len([f for f in os.listdir(train_folder) if os.path.isfile(os.path.join(train_folder, f))]) if os.path.exists(train_folder) else 0
        test_count = len([f for f in os.listdir(test_folder) if os.path.isfile(os.path.join(test_folder, f))]) if os.path.exists(test_folder) else 0

        print(f"Cartella '{folder}':")
        print(f"  Train set: {train_count} file")
        print(f"  Test set: {test_count} file")

Cartella 'eyes':
  Train set: 18255 file
  Test set: 4776 file
Cartella 'nose':
  Train set: 18255 file
  Test set: 4776 file
Cartella 'mouth':
  Train set: 18255 file
  Test set: 4776 file


In [4]:
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, Dataset
import matplotlib.pyplot as plt
import numpy as np
import torch

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [5]:
# Impostare la trasformazione
transform_eyes = transforms.Compose([
    transforms.Resize((44, 266)),  # Cambia dimensione a (altezza, larghezza)
    transforms.ToTensor(),        # Converte l'immagine in tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalizzazione
])

# Transform per il naso (108x170)
transform_nose = transforms.Compose([
    transforms.Resize((170, 108)),  # Cambia dimensione a (altezza, larghezza)
    transforms.ToTensor(),         # Converte l'immagine in tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalizzazione
])

# Transform per la bocca (193x89)
transform_mouth = transforms.Compose([
    transforms.Resize((89, 193)),  # Cambia dimensione a (altezza, larghezza)
    transforms.ToTensor(),        # Converte l'immagine in tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalizzazione
])

# Dataset personalizzato
class EthnicityDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []

        # Scansione della cartella per ottenere i file
        for filename in os.listdir(root_dir):
            if filename.endswith(".jpg") or filename.endswith(".png"):  # Considerando immagini .jpg e .png
                self.images.append(filename)

                # Estraiamo l'etnia dalla parte finale del nome del file
                # esempio: eyes_100_1_0_20170112213001988
                parts = filename.split('_')
                if len(parts) >= 4:
                    label = parts[3]  # L'etnia è il quarto elemento (indice 3)
                    label_map = {"0": 0, "1": 1, "2": 2, "3": 3, "4": 4}  # Mappatura delle etnie
                    self.labels.append(label_map.get(label, -1))  # -1 se l'etnia non è trovata

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_name = self.images[idx]
        image_path = os.path.join(self.root_dir, image_name)

        try:
            image = Image.open(image_path)
        except Exception as e:
            print(f"Errore nel caricamento dell'immagine {image_path}: {e}")
            return None, None

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]

        # Aggiungi un controllo se l'etichetta è corretta
        if label == -1:
            print(f"Errore: etichetta non trovata per {image_name}")

        return image, label

In [6]:
from Dataset.eyes_cnn import EyesCNN
from Dataset.nose_cnn import NoseCNN
from Dataset.mouth_cnn import MouthCNN

eyes_net = EyesCNN()
nose_net = NoseCNN()
mouth_net = MouthCNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
eyes_optimizer = optim.Adam(eyes_net.parameters(), lr=0.001)
nose_optimizer = optim.Adam(nose_net.parameters(), lr=0.001)
mouth_optimizer = optim.Adam(mouth_net.parameters(), lr=0.001)

**EYES TRAINING VALIDATION AND TEST**

In [7]:
# Inizializzare il dataset
full_train_dataset = EthnicityDataset(root_dir="Dataset/eyes/train", transform=transform_eyes)
test_dataset = EthnicityDataset(root_dir="Dataset/eyes/test", transform=transform_eyes)

#####################################################
Train_size = int(0.8 * len(full_train_dataset))
Val_size = len(full_train_dataset) - Train_size
train_dataset, val_dataset = random_split(full_train_dataset, [Train_size, Val_size])

#####################################################

# use these names for the data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

print(len(train_dataset), len(test_dataset), len(val_dataset))

14604 4776 3651


In [8]:
# Initialize lists to store metrics
eyes_train_losses = []
eyes_val_losses = []
eyes_train_accuracies = []
eyes_val_accuracies = []


num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    total_batches = 0

    # Training Phase
    eyes_net.train()
    correct = 0
    total = 0

    for data in train_loader:
        if data is None:
            continue  # Se il dato è None (errore nel caricamento), lo saltiamo

        inputs, labels = data

        # Zero the parameter gradients
        eyes_optimizer.zero_grad()

        # Forward pass
        outputs = eyes_net(inputs)
        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        eyes_optimizer.step()

        running_loss += loss.item()
        total_batches += 1

        # Calculate training accuracy
        _, y_pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (y_pred == labels).sum().item()

    # Calcolare la loss e l'accuratezza media per l'epoch
    avg_train_loss = running_loss / total_batches
    train_accuracy = correct / total

    print(f'Epoch {epoch + 1}, Average Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}')

    # Validation Phase
    eyes_net.eval()
    val_running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for val_data in val_loader:
            val_inputs, val_labels = val_data

            # Forward pass
            val_outputs = eyes_net(val_inputs)

            # Compute loss
            val_loss = criterion(val_outputs, val_labels)
            val_running_loss += val_loss.item()

            # Calculate accuracy
            _, val_predicted = torch.max(val_outputs.data, 1)
            total += val_labels.size(0)
            correct += (val_predicted == val_labels).sum().item()

    # Calculate average loss for the validation epoch
    avg_val_loss = val_loss / val_running_loss
    eyes_val_losses.append(avg_val_loss)

    # Calculate validation accuracy
    val_accuracy = correct / total
    eyes_val_accuracies.append(val_accuracy)


    print(f'Epoch {epoch + 1}, Average Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

print('Finished Training')

Epoch 1, Average Training Loss: 1.1682, Training Accuracy: 0.5498
Epoch 1, Average Validation Loss: 0.0248, Validation Accuracy: 0.6018
Epoch 2, Average Training Loss: 1.0139, Training Accuracy: 0.6259
Epoch 2, Average Validation Loss: 0.0080, Validation Accuracy: 0.6343
Epoch 3, Average Training Loss: 0.9696, Training Accuracy: 0.6435
Epoch 3, Average Validation Loss: 0.0260, Validation Accuracy: 0.6335
Epoch 4, Average Training Loss: 0.9408, Training Accuracy: 0.6553
Epoch 4, Average Validation Loss: 0.0136, Validation Accuracy: 0.6431
Epoch 5, Average Training Loss: 0.9213, Training Accuracy: 0.6602
Epoch 5, Average Validation Loss: 0.0082, Validation Accuracy: 0.6601
Epoch 6, Average Training Loss: 0.9031, Training Accuracy: 0.6736
Epoch 6, Average Validation Loss: 0.0157, Validation Accuracy: 0.6486
Epoch 7, Average Training Loss: 0.8901, Training Accuracy: 0.6775
Epoch 7, Average Validation Loss: 0.0066, Validation Accuracy: 0.6483
Epoch 8, Average Training Loss: 0.8724, Training

In [9]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = eyes_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test images: {100 * correct / total:.2f}%')

Accuracy on the test images: 65.05%


**NOSE TRAINING VALIDATION AND TEST**

In [10]:
# Inizializzare il dataset
full_train_dataset = EthnicityDataset(root_dir="Dataset/nose/train", transform=transform_nose)
test_dataset = EthnicityDataset(root_dir="Dataset/nose/test", transform=transform_nose)

#####################################################
Train_size = int(0.8 * len(full_train_dataset))
Val_size = len(full_train_dataset) - Train_size
train_dataset, val_dataset = random_split(full_train_dataset, [Train_size, Val_size])

#####################################################

# use these names for the data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

print(len(train_dataset), len(test_dataset), len(val_dataset))

14604 4776 3651


In [11]:
# Initialize lists to store metrics
nose_train_losses = []
nose_val_losses = []
nose_train_accuracies = []
nose_val_accuracies = []


num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    total_batches = 0

    # Training Phase
    nose_net.train()
    correct = 0
    total = 0

    for data in train_loader:
        if data is None:
            continue  # Se il dato è None (errore nel caricamento), lo saltiamo

        inputs, labels = data

        # Zero the parameter gradients
        nose_optimizer.zero_grad()

        # Forward pass
        outputs = nose_net(inputs)
        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        nose_optimizer.step()

        running_loss += loss.item()
        total_batches += 1

        # Calculate training accuracy
        _, y_pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (y_pred == labels).sum().item()

    # Calcolare la loss e l'accuratezza media per l'epoch
    avg_train_loss = running_loss / total_batches
    train_accuracy = correct / total

    print(f'Epoch {epoch + 1}, Average Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}')

    # Validation Phase
    nose_net.eval()
    val_running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for val_data in val_loader:
            val_inputs, val_labels = val_data

            # Forward pass
            val_outputs = nose_net(val_inputs)

            # Compute loss
            val_loss = criterion(val_outputs, val_labels)
            val_running_loss += val_loss.item()

            # Calculate accuracy
            _, val_predicted = torch.max(val_outputs.data, 1)
            total += val_labels.size(0)
            correct += (val_predicted == val_labels).sum().item()

    # Calculate average loss for the validation epoch
    avg_val_loss = val_loss / val_running_loss
    nose_val_losses.append(avg_val_loss)

    # Calculate validation accuracy
    val_accuracy = correct / total
    nose_val_accuracies.append(val_accuracy)


    print(f'Epoch {epoch + 1}, Average Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

print('Finished Training')

Epoch 1, Average Training Loss: 1.2725, Training Accuracy: 0.5008
Epoch 1, Average Validation Loss: 0.0166, Validation Accuracy: 0.5314
Epoch 2, Average Training Loss: 1.1654, Training Accuracy: 0.5522
Epoch 2, Average Validation Loss: 0.0256, Validation Accuracy: 0.5681
Epoch 3, Average Training Loss: 1.1187, Training Accuracy: 0.5737
Epoch 3, Average Validation Loss: 0.0121, Validation Accuracy: 0.5475
Epoch 4, Average Training Loss: 1.0916, Training Accuracy: 0.5864
Epoch 4, Average Validation Loss: 0.0187, Validation Accuracy: 0.5793
Epoch 5, Average Training Loss: 1.0678, Training Accuracy: 0.5972
Epoch 5, Average Validation Loss: 0.0061, Validation Accuracy: 0.5763
Epoch 6, Average Training Loss: 1.0629, Training Accuracy: 0.6005
Epoch 6, Average Validation Loss: 0.0303, Validation Accuracy: 0.5982
Epoch 7, Average Training Loss: 1.0366, Training Accuracy: 0.6118
Epoch 7, Average Validation Loss: 0.0267, Validation Accuracy: 0.5812
Epoch 8, Average Training Loss: 1.0203, Training

In [12]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = nose_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test images: {100 * correct / total:.2f}%')

Accuracy on the test images: 59.86%


**MOUTH TRAINING VALIDATION AND TEST**

In [13]:
# Inizializzare il dataset
full_train_dataset = EthnicityDataset(root_dir="Dataset/mouth/train", transform=transform_mouth)
test_dataset = EthnicityDataset(root_dir="Dataset/mouth/test", transform=transform_mouth)

#####################################################
Train_size = int(0.8 * len(full_train_dataset))
Val_size = len(full_train_dataset) - Train_size
train_dataset, val_dataset = random_split(full_train_dataset, [Train_size, Val_size])

#####################################################

# use these names for the data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

print(len(train_dataset), len(test_dataset), len(val_dataset))

14604 4776 3651


In [14]:
# Initialize lists to store metrics
mouth_train_losses = []
mouth_val_losses = []
mouth_train_accuracies = []
mouth_val_accuracies = []


num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    total_batches = 0

    # Training Phase
    mouth_net.train()
    correct = 0
    total = 0

    for data in train_loader:
        if data is None:
            continue  # Se il dato è None (errore nel caricamento), lo saltiamo

        inputs, labels = data

        # Zero the parameter gradients
        mouth_optimizer.zero_grad()

        # Forward pass
        outputs = mouth_net(inputs)
        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        mouth_optimizer.step()

        running_loss += loss.item()
        total_batches += 1

        # Calculate training accuracy
        _, y_pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (y_pred == labels).sum().item()

    # Calcolare la loss e l'accuratezza media per l'epoch
    avg_train_loss = running_loss / total_batches
    train_accuracy = correct / total

    print(f'Epoch {epoch + 1}, Average Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}')

    # Validation Phase
    mouth_net.eval()
    val_running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for val_data in val_loader:
            val_inputs, val_labels = val_data

            # Forward pass
            val_outputs = mouth_net(val_inputs)

            # Compute loss
            val_loss = criterion(val_outputs, val_labels)
            val_running_loss += val_loss.item()

            # Calculate accuracy
            _, val_predicted = torch.max(val_outputs.data, 1)
            total += val_labels.size(0)
            correct += (val_predicted == val_labels).sum().item()

    # Calculate average loss for the validation epoch
    avg_val_loss = val_loss / val_running_loss
    mouth_val_losses.append(avg_val_loss)

    # Calculate validation accuracy
    val_accuracy = correct / total
    mouth_val_accuracies.append(val_accuracy)


    print(f'Epoch {epoch + 1}, Average Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

print('Finished Training')

Epoch 1, Average Training Loss: 1.1753, Training Accuracy: 0.5503
Epoch 1, Average Validation Loss: 0.0178, Validation Accuracy: 0.5420
Epoch 2, Average Training Loss: 1.1076, Training Accuracy: 0.5813
Epoch 2, Average Validation Loss: 0.0125, Validation Accuracy: 0.5664
Epoch 3, Average Training Loss: 1.0797, Training Accuracy: 0.5921
Epoch 3, Average Validation Loss: 0.0181, Validation Accuracy: 0.5776
Epoch 4, Average Training Loss: 1.0590, Training Accuracy: 0.6011
Epoch 4, Average Validation Loss: 0.0231, Validation Accuracy: 0.5867
Epoch 5, Average Training Loss: 1.0399, Training Accuracy: 0.6070
Epoch 5, Average Validation Loss: 0.0212, Validation Accuracy: 0.5774
Epoch 6, Average Training Loss: 1.0168, Training Accuracy: 0.6175
Epoch 6, Average Validation Loss: 0.0226, Validation Accuracy: 0.5831
Epoch 7, Average Training Loss: 0.9984, Training Accuracy: 0.6277
Epoch 7, Average Validation Loss: 0.0176, Validation Accuracy: 0.5889
Epoch 8, Average Training Loss: 0.9910, Training

In [15]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = mouth_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test images: {100 * correct / total:.2f}%')

Accuracy on the test images: 60.03%
